In [ ]:
!pip install qdrant-client

In [ ]:
!pip install -U transformers


In [ ]:
import qdrant_client
from qdrant_client.http import models
import os
import uuid

In [ ]:
# create your client

os.environ['QDRANT_HOST'] = 'your host link on qdrant'
os.environ['QDRANT_API_KEY'] = 'your api key'


client = qdrant_client.QdrantClient(url=
        os.getenv("QDRANT_HOST"),
        api_key=os.getenv("QDRANT_API_KEY")
    )

In [ ]:
# create collection

os.environ['QDRANT_COLLECTION'] = "testcollections"

collection_config = models.VectorParams(
        size=384, # 768 for instructor-xl, 1536 for OpenAI
        distance=models.Distance.COSINE
    )

client.recreate_collection(
    collection_name=os.getenv("QDRANT_COLLECTION"),
    vectors_config=collection_config
)

True

In [ ]:
!pip install torch

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import numpy as np
model = AutoModel.from_pretrained("BAAI/bge-small-en-v1.5")
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-small-en-v1.5")


In [ ]:
text =  "healthcare chatbots in nlp"
def query_emd(query):
  encoded_text = tokenizer(query, return_tensors="pt")
  with torch.no_grad():
      model_output = model(**encoded_text)
      embeddings = model_output.pooler_output
  embeddings_array = embeddings.detach().numpy()
  return embeddings_array

query_embed = query_emd(text)
print(query_embed.shape)
# print(query_embed)
#embeddings_array.shape

(1, 384)


In [ ]:
# to create the embeddings for each documents

def document_embeddings(documents):
    embedded_documents = []
    for doc in documents:
        encoded_text = tokenizer(doc, return_tensors="pt")  # returns a tensor of 3-D and higher dimension
        with torch.no_grad():
            model_output = model(**encoded_text)
            embeddings = model_output.pooler_output
        embeddings_array = embeddings.detach().numpy()  #converting it into numoy array for processing
        embedded_documents.append(embeddings_array)

    # Concatenate the embeddings along the first axis (documents)
    # document_embeddings = np.concatenate(embedded_documents, axis=0)
    # document_embeddings = np.vstack(embedded_documents) # converting into 2-D for similarity
    return embedded_documents


In [ ]:
documents = [
    "Science Fiction: In the year 2150, humanity has colonized Mars. The Martian colonies thrive under the domes protecting them from the harsh environment outside. However, tensions rise between Earth and Mars as resources become scarce. A group of scientists discovers a way to terraform Mars, but it comes with a risk of irreversible consequences. As the fate of two worlds hangs in the balance, a young engineer must navigate political intrigue and personal sacrifice to save both planets from destruction.",

    "Fantasy: In the kingdom of Eldoria, magic is a forbidden art. The tyrannical king has outlawed all forms of sorcery, fearing its power. But when a young orphan discovers she has the gift of magic, she becomes the key to overthrowing the king's reign of terror. With the help of a band of rebels and a wise old wizard, she embarks on a quest to reclaim the kingdom's lost magic and restore balance to the land.",

    "Mystery: Detective John Smith is called to investigate a series of gruesome murders in the small town of Rivertown. As he delves deeper into the case, he uncovers a web of lies, deceit, and betrayal. With the clock ticking and the body count rising, Detective Smith must race against time to catch the killer before they strike again.",

    "Romance: Emma and James have been best friends since childhood, but as they grow older, their relationship begins to evolve into something more. Caught between friendship and love, they must navigate the complexities of their feelings while facing challenges and obstacles along the way. Will they risk it all for a chance at true love?",

    "Historical Fiction: Set during World War II, 'The Last Letter' follows the story of a young soldier deployed to the front lines. Through letters exchanged with his sweetheart back home, he shares his experiences, fears, and hopes for the future. As the war rages on, their love is put to the test, but their bond remains unbreakable.",

    "Thriller: Sarah's life is turned upside down when she receives a mysterious package containing a cryptic message and a photograph of a crime scene. Desperate for answers, she embarks on a dangerous journey to uncover the truth behind the photograph. But as she gets closer to the truth, she realizes that someone is watching her every move, and they will stop at nothing to keep their secrets hidden.",

    "Horror: A group of friends ventures into the abandoned asylum on the outskirts of town, seeking thrills and adventure. But they soon discover that they are not alone in the dark corridors of the asylum. Something sinister lurks in the shadows, stalking them one by one. As they struggle to escape, they realize that they may have unleashed an ancient evil that cannot be stopped.",

    "Adventure: Captain Jack and his crew set sail on a daring quest to find the lost treasure of the legendary pirate Blackbeard. Battling treacherous seas, rival pirates, and mythical creatures, they must brave the dangers of the open ocean to claim the riches that lie buried beneath the waves.",

    "Drama: In the bustling city of New York, four strangers' lives intersect in unexpected ways. From a struggling artist to a successful businessman, each character faces their own challenges and desires as they navigate the complexities of love, ambition, and betrayal in the urban jungle.",

    "Comedy: When two rival comedians are forced to work together on a new comedy special, chaos ensues. With clashing egos and conflicting comedic styles, they must put aside their differences to deliver the performance of a lifetime. But as tensions rise and tempers flare, will they be able to set aside their differences and make audiences laugh?",

    "Artificial intelligence is taking  over tha world in a fast pace. the natural language processing and machine learning play a vital role in the healtcare fields for creating personalized chatbots and models to detect the medical issue in the patients in order to provide a personalized healthcare support to thE patients thereby reducing the workload loads of the doctors."
]


In [ ]:
doc_emd = document_embeddings(documents)
ans = np.array(doc_emd)
print(ans.shape)

(11, 1, 384)


In [ ]:
# Define a function to generate unique IDs
def generate_unique_id():
    return str(uuid.uuid4())

In [ ]:
# Insert each document along with its embedding into the collection
docs = {}
for i, doc in enumerate(documents):
    document_id = generate_unique_id()  # Generate a unique ID for each document
    embedding = document_embeddings(documents[i])
    docs[document_id] = documents[i]

    client.upsert(
        collection_name = os.environ['QDRANT_COLLECTION'],
        points = models.Batch
         (
            ids = [document_id],
            vectors = embedding[0]
          )
    )

In [ ]:
search = client.search(
    collection_name=os.environ['QDRANT_COLLECTION'],
    search_params=models.SearchParams(hnsw_ef=128, exact=False),
    query_vector=query_embed[0],
    limit=3
)
print(search)


[ScoredPoint(id='7cf71a90-b704-4f8a-b684-3bcb65be8f55', version=10, score=0.57098305, payload={}, vector=None, shard_key=None), ScoredPoint(id='509a4677-e5cb-405d-8ed0-cb458fb2a4cf', version=7, score=0.57098305, payload={}, vector=None, shard_key=None), ScoredPoint(id='efe523e4-c5be-40b3-b822-2083cb742838', version=3, score=0.55894995, payload={}, vector=None, shard_key=None)]


In [ ]:
document_ids = [result.id for result in search]

# Printing the IDs
for doc_id in document_ids:
    print("Document ID:", doc_id)

Document ID: 7cf71a90-b704-4f8a-b684-3bcb65be8f55
Document ID: 509a4677-e5cb-405d-8ed0-cb458fb2a4cf
Document ID: efe523e4-c5be-40b3-b822-2083cb742838


In [ ]:
print("The similar sentences with highest scores is")
for i in range(len(document_ids)):
  print(f"The document in the place {i+1} is:  ")
  print(docs[document_ids[i]])

The similar sentences with highest scores is
The document in the place 1 is:  
Artificial intelligence is taking  over tha world in a fast pace. the natural language processing and machine learning play a vital role in the healtcare fields for creating personalized chatbots and models to detect the medical issue in the patients in order to provide a personalized healthcare support to thE patients thereby reducing the workload loads of the doctors.
The document in the place 2 is:  
Adventure: Captain Jack and his crew set sail on a daring quest to find the lost treasure of the legendary pirate Blackbeard. Battling treacherous seas, rival pirates, and mythical creatures, they must brave the dangers of the open ocean to claim the riches that lie buried beneath the waves.
The document in the place 3 is:  
Romance: Emma and James have been best friends since childhood, but as they grow older, their relationship begins to evolve into something more. Caught between friendship and love, they m